In [163]:
"""
Author: Seung Won Joeng
Modifier: Kwanyeob Jung, Jaeshin Cho
"""
import numpy as np
import pandas as pd
import datetime as dt
%matplotlib inline                                  
import matplotlib.pyplot as plt
np.random.seed(1234)


In [164]:
# Read csv from current path.
def read_csv():
    df1 = pd.read_csv('2020_US_weekly_symptoms_dataset.csv')
    df2 = pd.read_csv('aggregated_cc_by.csv')
    return df1, df2

df1, df2 = read_csv()
print('Type of df1: ', type(df1))
print('Type of df2: ', type(df2))
print('Shape of df1: ', df1.shape);
print('Shape of df2: ', df2.shape);

Type of df1:  <class 'pandas.core.frame.DataFrame'>
Type of df2:  <class 'pandas.core.frame.DataFrame'>
Shape of df1:  (624, 430)
Shape of df2:  (98434, 62)


/Users/stevenjoeng/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [165]:
# Since we are going to handle regions in USA from 2020_US_weekly_symptoms_dataset.csv
# To extract the records of USA only from dataframe
def extract_from_aggregated(df1, df2):
    # Get all region codes 
    region_codes = df1.open_covid_region_code.unique()
    result = df2[df2['open_covid_region_code'].isin(region_codes)]
    result = result.reset_index(drop = True)
    result = result[['open_covid_region_code', 'region_name', 'date', 'hospitalized_new']]
    return result

df2 = extract_from_aggregated(df1, df2)
print('Shape of df2: ', df2.shape);
print(df2.head(10))


Shape of df2:  (3458, 4)
  open_covid_region_code region_name        date  hospitalized_new
0                  US-WY     Wyoming  2020/03/07               0.0
1                  US-WY     Wyoming  2020/03/08               0.0
2                  US-WY     Wyoming  2020/03/09               0.0
3                  US-WY     Wyoming  2020/03/10               0.0
4                  US-WY     Wyoming  2020/03/11               0.0
5                  US-WY     Wyoming  2020/03/12               0.0
6                  US-WY     Wyoming  2020/03/13               0.0
7                  US-WY     Wyoming  2020/03/14               0.0
8                  US-WY     Wyoming  2020/03/15               0.0
9                  US-WY     Wyoming  2020/03/16               0.0


In [171]:
# Clean data in threshold num_rows * 0.05 and num_cols * 0.05
def clean_dataframe(df):
    num_rows,num_cols = df1.shape
    thresh_rows = int(num_rows * 0.03);
    thresh_cols = int(num_cols * 0.03);

    df = df.dropna(axis = 1, thresh = thresh_rows)
    df = df.dropna(axis = 0, thresh = thresh_cols)
    df = df.reset_index(drop = True)
    return df;

df1 = clean_dataframe(df1)
df2 = clean_dataframe(df2)

print('Shape of df1: ', df1.shape);
print('Shape of df2: ', df2.shape);
print(df2)

Shape of df1:  (496, 127)
Shape of df2:  (3458, 4)
     open_covid_region_code region_name        date  hospitalized_new
0                     US-WY     Wyoming  2020/03/07               0.0
1                     US-WY     Wyoming  2020/03/08               0.0
2                     US-WY     Wyoming  2020/03/09               0.0
3                     US-WY     Wyoming  2020/03/10               0.0
4                     US-WY     Wyoming  2020/03/11               0.0
...                     ...         ...         ...               ...
3453                  US-AK      Alaska  2020/10/03               0.0
3454                  US-AK      Alaska  2020/10/04               0.0
3455                  US-AK      Alaska  2020/10/05               0.0
3456                  US-AK      Alaska  2020/10/06               0.0
3457                  US-AK      Alaska  2020/10/07               0.0

[3458 rows x 4 columns]


In [198]:
"""
Function merge_data takes two dataframes and merge two dataframes as one dataframe with weekly records.
Parameter df1 corresponds to 2020_US_weekly_symptoms_dataset.csv which is weekly records
Parameter df2 corresponds to aggregated_cc_by.csv which is daily records
"""

def same_week(d1, d2):
#     d1 = datetime.datetime.strptime(date1,'%Y-%m-%d')
#     d2 = datetime.datetime.strptime(date2,'%Y-%m-%d')
    return (d1.isocalendar()[1] == d2.isocalendar()[1])

def convert_datetime(df1, df2):
    df1['date'] = pd.to_datetime(df1.date)
    df2['date'] = pd.to_datetime(df2.date)
    return df1,df2


# d1 = '2020-03-09'
# d2 = '2020-03-16'
# print(same_week(d1,d2))


def daily_to_weekly(df2):
    df2 = df2.groupby(['open_covid_region_code', pd.Grouper(key='date', freq='1W')])['hospitalized_new'].sum().reset_index().sort_values(['open_covid_region_code', 'date'])
    return df2


def merge_data(df1, df2):
    


df1, df2 = convert_datetime(df1,df2)
print(df2['date'])
print('--------------------------------')
df2 = daily_to_weekly(df2)
print(df2['date'])
print('--------------------------------')
print(df2.date[0])
print(df1.date[7])
print(same_week(df1.date[7], df2.date[0]))
print('--------------------------------')


0     2020-03-01
1     2020-03-08
2     2020-03-15
3     2020-03-22
4     2020-03-29
         ...    
508   2020-09-06
509   2020-09-13
510   2020-09-20
511   2020-09-27
512   2020-10-04
Name: date, Length: 513, dtype: datetime64[ns]
--------------------------------
0     2020-03-01
1     2020-03-08
2     2020-03-15
3     2020-03-22
4     2020-03-29
         ...    
508   2020-09-06
509   2020-09-13
510   2020-09-20
511   2020-09-27
512   2020-10-04
Name: date, Length: 513, dtype: datetime64[ns]
--------------------------------
2020-03-01 00:00:00
2020-02-24 00:00:00
True
